## Auto mode

This is for testing propouses

In [1]:
from config import Color
from helpers import clean  #, local2global
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigQGC,
    ConfigVis,
    Gazebo,
    NoneVisualizer,
    Simulator,
)

clean()

## Simulation Configuration

In [ ]:
gra_origin = GRAPose(-35.3633280, 149.1652241, 0, 90)   # east, north, up, heading
enu_origin = ENUPose(0, 0, gra_origin.alt, gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 15., 0., 0.),
    (15., 0., 0., 0),
    (5., -20., 0., 30.),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),

])
base_paths = [Plan.create_square_path(side_len=10, alt=5,heading=0) for _ in base_homes]

colors=[
    Color.BLUE,
    Color.GREEN,
    Color.BLACK,
    Color.ORANGE,
    Color.RED,
]
msn_delays=[0,1,2,3,4] # in seconds

## Gazebo Congiguration

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
gaz_config.show()

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

## No Simulator

In [5]:
novis_config = ConfigVis[int]()

for i,_ in enumerate(base_homes):
    novis_config.add_vehicle(i)

## Save the missions

In [6]:
qgc_config.save_missions()

## Visualization Parameters

In [7]:
novis=NoneVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulatotion

In [8]:
simulator = Simulator(
	visualizers=[gaz],
	verbose=2,
)
orac = simulator.launch(gcs_sysids={'blue 🟦': [1,2,3,4,5]})

🖥️ Gazebo launched for realistic simulation and 3D visualization.
🚀 ArduPilot SITL vehicle 1 launched (PID 1385240)
🚀 ArduPilot SITL vehicle 2 launched (PID 1385241)
🚀 ArduPilot SITL vehicle 3 launched (PID 1385243)
🚀 ArduPilot SITL vehicle 4 launched (PID 1385245)
🚀 ArduPilot SITL vehicle 5 launched (PID 1385246)
🚀 UAV logic for vehicle 1 launched (PID 1385249)
🚀 UAV logic for vehicle 3 launched (PID 1385251)
🚀 UAV logic for vehicle 2 launched (PID 1385250)
🚀 UAV logic for vehicle 4 launched (PID 1385252)
🚀 UAV logic for vehicle 5 launched (PID 1385253)
🚀 Proxy for vehicle 1 launched (PID 1385257)
🔗 UAV logic 1 is connected to Ardupilot SITL vehicle 1
🚀 Proxy for vehicle 3 launched (PID 1385255)
🔗 UAV logic 3 is connected to Ardupilot SITL vehicle 3
🚀 Proxy for vehicle 2 launched (PID 1385258)
🔗 UAV logic 2 is connected to Ardupilot SITL vehicle 2
🚀 Proxy for vehicle 4 launched (PID 1385264)
🔗 UAV logic 4 is connected to Ardupilot SITL vehicle 4
🚀 Proxy for vehicle 5 launched (PID 138

bind port 5790 for SERIAL0
SERIAL0 on TCP port 5790
bind port 5770 for SERIAL0
SERIAL0 on TCP port 5770
bind port 5780 for SERIAL0
SERIAL0 on TCP port 5780
bind port 5800 for SERIAL0
SERIAL0 on TCP port 5800
bind port 5760 for SERIAL0
SERIAL0 on TCP port 5760


System id: 3
System id: 3
Connection on serial port 5780

🚀 Starting Proxy 3
System id: 5
Connection on serial port 5800
[Errno 111] Connection refused sleeping
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
System id: 1
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
System id: 2
System id: 5
System id: 2
Connection on serial port 5770

🚀 Starting Proxy 2
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
System id: 1
Connection on serial port 5760

🚀 Starting Proxy 1
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/auto

Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.


Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/def

bind port 5782 for SERIAL1
SERIAL1 on TCP port 5782
bind port 5783 for SERIAL2
SERIAL2 on TCP port 5783
bind port 5792 for SERIAL1
SERIAL1 on TCP port 5792
bind port 5793 for SERIAL2
SERIAL2 on TCP port 5793
validate_structures:528: Validating structures
validate_structures:528: Validating structures
bind port 5772 for SERIAL1
SERIAL1 on TCP port 5772
bind port 5773 for SERIAL2
SERIAL2 on TCP port 5773
bind port 5762 for SERIAL1
SERIAL1 on TCP port 5762
bind port 5763 for SERIAL2
SERIAL2 on TCP port 5763
validate_structures:528: Validating structures
validate_structures:528: Validating structures
bind port 5802 for SERIAL1
SERIAL1 on TCP port 5802
bind port 5803 for SERIAL2
SERIAL2 on TCP port 5803
validate_structures:528: Validating structures


Vehicle 5: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 5: ▶️ Step Started: clear uav missions


## Oracle checking

In [9]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
SIM_VEHICLE: Waiting for SITL to exit
🔗 UAV logic 1 is connected
🔗 UAV logic 2 is connected
Vehicle 4: 🧹 Cleared previous mission
Vehicle 4: ✅ Step Done: clear uav missions
Vehicle 3: 🧹 Cleared previous mission
Vehicle 3: ✅ Step Done: clear uav missions
Vehicle 1: 🧹 Cleared previous mission
Vehicle 1: ✅ Step Done: clear uav missions
Vehicle 2: 🧹 Cleared previous mission
Vehicle 2: ✅ Step Done: clear uav missions
Vehicle 5: 🧹 Cleared previous mission
Vehicle 5: ✅ Step Done: clear uav missions
Vehicle 4: ✅ 8 waypoints read.
Vehicle 4: 🧭 Mission[0] → cmd: WAYPOINT, x: -35.3634627, y: 149.1653893, z: 0.0, current: 0
Vehicle 4: 🧭 Mission[1] → cmd: DELAY, x: -35.3634627, y: 149.1653893, z: 0.0, current: 0
Vehicle 4: 🧭 Mission[2] → cmd: TAKEOFF, x: -35.3634627, y: 149.1653893, z: 5.0, current: 0
Vehicle 4: 🧭 Mission[3] → cmd: WAYPOINT, x: -35.

✅ ACK received. DONE message acknowledged.
❎ Vehicle 5 logic stopped.
Connection reset or closed by peer on TCP socket
💾 Trajectories saved to 'trajectories_blue 🟦.pkl'.
❎ Proxy 5 stopped.
Closed connection on SERIAL0
